### DEV


In [11]:
from pathlib import Path
import sys
import os
import django
import pandas as pd
from asgiref.sync import sync_to_async

# Add the parent directory to the system path and setup django
BASE_DIR = str(Path(os.getcwd()).resolve().parent.parent)

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend_site.settings")

django.setup()

from tsosi.models import Entity, Transfer
from django.utils import timezone
from django.db.models import Q


@sync_to_async
def run():
    fields = ["id", "wikipedia_url", "wikipedia_extract"]
    date_value = timezone.now() - timezone.timedelta(days=15)
    date_condition = Q(date_wikipedia_fetched__isnull=True) | Q(
        date_wikipedia_fetched__lt=date_value
    )
    instances = (
        Entity.objects.filter(is_active=True, wikipedia_url__isnull=False)
        .filter(date_condition)
        .filter(date_wikipedia_fetched__isnull=False)
        .values(
            "id", "wikipedia_url", "wikipedia_extract", "date_wikipedia_fetched"
        )
    )
    df = pd.DataFrame.from_records(instances)
    for c in fields:
        df[c] = None

    return df


res = await run()

In [12]:
res

,id,wikipedia_url,wikipedia_extract


In [ ]:
from pycountry import countries

d = (
    res.groupby("country")
    .count()
    .reset_index()
    .sort_values("id", ascending=False)
)
mapping = {c.alpha_2: c.name for c in countries}
d["country_name"] = d["country"].map(mapping)
d.rename(columns={"country": "country_code", "id": "number"}, inplace=True)


categories = [(50, "Upper"), (10, "Medium"), (0, "Lower")]

for c_threshold, c_name in sorted(categories, key=lambda x: x[0]):
    mask = d["number"] >= c_threshold
    d.loc[mask, "category"] = c_name

d[["country_name", "country_code", "number", "category"]].to_excel(
    "supporters_extract.xlsx", index=False
)

In [ ]:
sorted(categories, key=lambda x: x[0])

In [ ]:
from fontTools.ttLib.woff2 import compress
from pathlib import Path
import re

font_directory = Path.home() / "Documents/TSOSI/Graphisme/Mont/Mont"
otf_files = font_directory.glob("*.otf")
save_directory = Path.home() / "Documents/TSOSI/Fonts/Mont"

for file in otf_files:
    file_path_usefull = re.match(r".*(Mont[^\.]+)", file.name)
    save_path = (
        str(save_directory / file_path_usefull[1]).replace(" ", "_") + ".woff2"
    )
    compress(str(file), str(save_path))

In [ ]:
import requests

query_base = """
SELECT
    ?item
    ?itemLabel
    ?itemDescription
    ?countryIsoCode
    ?logoUrl
    ?websiteUrl
    ?rorId
    ?wikipediaUrl
    ?coordinates
    ?inceptionDate
    ?statementCount
WHERE {{
    VALUES ?item {{ {ids_part} }}
    OPTIONAL {{ ?item wdt:P571 ?inceptionDate. }}
    OPTIONAL {{ ?item (wdt:P17/wdt:P297) ?countryIsoCode. }}
    OPTIONAL {{ ?item wdt:P154 ?logoUrl. }}
    OPTIONAL {{ ?item wdt:P856 ?websiteUrl. }}
    OPTIONAL {{ ?item wdt:P6782 ?rorId. }}
    OPTIONAL {{ ?item wdt:P625 ?coordinates. }}
    OPTIONAL {{ ?item wikibase:statements ?statementCount. }}
    OPTIONAL {{
        ?wikipediaUrl rdf:type schema:Article;
        schema:about ?item;
        schema:inLanguage "en";
        schema:isPartOf <https://en.wikipedia.org/>.
    }}
    SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
    }}
}}
"""

ids = ["Q215432154632121", "QNOT_EXISTING"]
query = query_base.format(ids_part="\n\t\t".join([f"wd:{id}" for id in ids]))
params = {"query": query, "format": "json"}

resp = requests.post("https://query.wikidata.org/sparql", params=params)
resp.json()["results"]["bindings"]

[{'item': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q215432154632121'},
  'itemLabel': {'type': 'literal', 'value': 'Q215432154632121'}},
 {'item': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/QNOT_EXISTING'},
  'itemLabel': {'type': 'literal', 'value': 'QNOT_EXISTING'}}]

In [1]:
import pandas as pd

df = pd.DataFrame()
# We only want wikipedia extract for entities with URL to the english wiki
mask = df["wikipedia_url"].str.startswith("https://en.wikipedia.org")
df[mask].reset_index(drop=True)

KeyError: 'wikipedia_url'